<center>
    
# Google Maps Location API

</center>

<div style="text-align: right; width: 15%; margin-left: auto;">
    
Jon Cannaday <br>
BI Team <br>
9/27/2024  

</div>

## Libraries
<br>

<div style="width: 100%; border-bottom: 1px solid black;"></div>

In [4]:
import googlemaps
import pandas as pd


## flat files
<br>

<div style="width: 100%; border-bottom: 1px solid black;"></div>

The flat file contains a list of theaters amoung other demographic data. The table was processed before being pulled into this work flow.

In [5]:
theater_data = pd.DataFrame(pd.read_excel(
    '...\\theater_data.xlsx')
) 

In [6]:
addresses = theater_data['address']
addresses

0                    3900 Dallas Pkwy, Plano, TX 75093
1               3471 Old Hwy 77, Brownsville, TX 78520
2            1848 E Stop 13 Rd, Indianapolis, IN 46227
3       5063 Northwest Loop 410, San Antonio, TX 78229
4                   4108 Burbank Rd, Wooster, OH 44691
                            ...                       
331                 2400 Creekview Dr., Waco, TX 76711
332    26602 Towne Centre Dr, Foothill Ranch, CA 92610
333            7301 W Grand Pkwy S, Richmond, TX 77407
334            28250 Diehl Rd a, Warrenville, IL 60555
335        9161 Commerce Center Dr, Columbia, MD 21046
Name: address, Length: 336, dtype: object


## Initiate google maps client
<br>

<div style="width: 100%; border-bottom: 1px solid black;"></div>

In [7]:
gmaps = googlemaps.Client(key="xxx")


## define variables
<br>

<div style="width: 100%; border-bottom: 1px solid black;"></div>

The plae types were retrieved from the google api documentation and the radius was set to 1 km from each theater location.

In [8]:
radius = 1000
place_types = [
    "accounting", "airport", "amusement_park", "aquarium", "art_gallery", "atm", "bakery", "bank", "bar",
    "beauty_salon", "bicycle_store", "book_store", "bowling_alley", "bus_station", "cafe", "campground",
    "car_dealer", "car_rental", "car_repair", "car_wash", "casino", "cemetery", "church", "city_hall",
    "clothing_store", "convenience_store", "courthouse", "dentist", "department_store", "doctor", "drugstore",
    "electrician", "electronics_store", "embassy", "fire_station", "florist", "funeral_home", "furniture_store",
    "gas_station", "gym", "hair_care", "hardware_store", "hindu_temple", "home_goods_store", "hospital",
    "insurance_agency", "jewelry_store", "laundry", "lawyer", "library", "light_rail_station", "liquor_store",
    "local_government_office", "locksmith", "lodging", "meal_delivery", "meal_takeaway", "mosque", "movie_rental",
    "movie_theater", "moving_company", "museum", "night_club", "painter", "park", "parking", "pet_store",
    "pharmacy", "physiotherapist", "plumber", "police", "post_office", "primary_school", "real_estate_agency",
    "restaurant", "roofing_contractor", "rv_park", "school", "secondary_school", "shoe_store", "shopping_mall",
    "spa", "stadium", "storage", "store", "subway_station", "supermarket", "synagogue", "taxi_stand",
    "tourist_attraction", "train_station", "transit_station", "travel_agency", "university", "veterinary_care", "zoo"
]

## data holding
<br>

<div style="width: 100%; border-bottom: 1px solid black;"></div>

In [9]:
all_data = []

## api
<br>

<div style="width: 100%; border-bottom: 1px solid black;"></div>

What will happen is, for each theater in the flat file, we will retrieve the names of different entities that are amoung each place type. Each cell will cantain either a blank value or a list of entities.

In [10]:
for address in addresses:
    try:
        # Geocode the address to get latitude and longitude
        geocode_result = gmaps.geocode(address)
        location = geocode_result[0]['geometry']['location']
        latitude = location['lat']
        longitude = location['lng']
        
        # Dictionary to hold places data for this address
        places_by_type = {}

        # Find nearby places for each place type
        for place_type in place_types:
            places_result = gmaps.places_nearby(location=(latitude, longitude), radius=radius, type=place_type)
            place_names = [place['name'] for place in places_result.get('results', [])]
            places_by_type[place_type] = ', '.join(place_names) if place_names else "No results"

        # Add the address to the result
        places_by_type['Address'] = address
        all_data.append(places_by_type)

    except Exception as e:
        print(f"Error processing address {address}: {e}")


## extraction
<br>

<div style="width: 100%; border-bottom: 1px solid black;"></div>

In [11]:
theater_features = pd.DataFrame(all_data)

In [15]:
theater_features

,accounting,airport,amusement_park,aquarium,art_gallery,atm,bakery,bank,bar,beauty_salon,...,synagogue,taxi_stand,tourist_attraction,train_station,transit_station,travel_agency,university,veterinary_care,zoo,Address
0,"Shamshiri CPA Group, Plano CPA Office, Debra A...",No results,No results,No results,Premium Arts Plano,"Coinstar Kiosk - Bitcoin ATM, Chase ATM, Chase...","Baskin-Robbins, Tom Thumb, Kroger Bakery, Eatz...","Chase Bank, Bank of America (with Drive-thru A...","Whiskey Cake Kitchen & Bar, Sixty Vines, Parkw...","HBG Salon, Serenity Salon, José Eber Salon, J....",...,No results,No results,"Coyote Creek Park, Windhaven Park",No results,"Communications @ Glenhollow - N - FS, Communic...",EW Select - Travel Concierge,No results,"Windhaven Veterinary Hospital, CityVet | Windh...",No results,"3900 Dallas Pkwy, Plano, TX 75093"
1,"Juan C. Mendoza, CPA, CICA, Carlos Barrera, CP...",No results,No results,No results,No results,"Sunrise Mall, Rio Bank - Brownsville Banking C...","Great American Cookies Sunrise, Krispy Kreme, ...","Rio Bank - Brownsville Banking Center, IBC Ban...","Shenanigan's Irish Pub & Grill, Gazpachos Rest...","JCPenney Salon, iLO Brow & Lash Studio, LASER ...",...,No results,No results,No results,No results,"FM 802 & Weslaco Rd., FM 802 (Across Lowes), S...",Sanborns Mexico Insurance,No results,No results,No results,"3471 Old Hwy 77, Brownsville, TX 78520"
2,"H&R Block, 1040 Family Taxes, Boyd Pinegar & A...",No results,FUNBOX INDIANAPOLIS - GREENWOOD,No results,No results,"FORUM Credit Union, BMO Bank N.A Branches, Cha...","Auntie Anne's, The Cheesecake Factory, Fresh T...","BMO Bank N.A Branches, Huntington Bank, Citize...","BJ's Restaurant & Brewhouse, Olive Garden Ital...",Hair Extensions & Braids and So Much More by A...,...,No results,No results,No results,No results,"US 31 & Stop 13 Rd, County Line Rd at Greenwoo...",#csirealtyteam #mywayvaca,No results,No results,No results,"1848 E Stop 13 Rd, Indianapolis, IN 46227"
3,"Texas Workforce Commission Tax Office, H&R Block",No results,No results,No results,No results,"Wells Fargo ATM, Coinstar Kiosk - Bitcoin ATM,...","Walmart Supercenter, Sam's Club, La Michoacana...",No results,"Chili's Grill & Bar, What's It To Ya Lounge, H...","Hair Xperts Salon, Sam's Nail supply, Melody H...",...,No results,No results,No results,No results,"Glen Ridge & Knights Bridge, Evers & Glen Ridg...",No results,No results,ShotVet @ PetSmart,No results,"5063 Northwest Loop 410, San Antonio, TX 78229"
4,"Whitaker Myers Tax Advisors, Ltd.",No results,No results,No results,Franklin Art & Design,"Farmers National Bank, The Commercial & Saving...","Walmart Supercenter, Panera Bread, Auntie Anne...","Farmers National Bank, The Commercial & Saving...","Applebee's Grill + Bar, LongHorn Steakhouse","Great Clips, Skin Care Solutions, Ulta Beauty,...",...,No results,No results,No results,No results,No results,"AAA Wooster, TEAMINN, LLC, Maibach Travel",No results,"Petco Vaccination Clinic, ShotVet @ PetSmart",No results,"4108 Burbank Rd, Wooster, OH 44691"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331,Bestor & Associates PC,No results,No results,No results,No results,"Cardtronics ATM, Stripes, ATM (Flying J Travel...",Nothing Bundt Cakes,No results,"Old Chicago Pizza + Taproom, Topgolf Waco","R & M Beauty Style, Tiffany's Nails, Hand and ...",...,No results,No results,No results,No results,"Central Texas Marketplace, WISD Sports Complex",Truly Your Destination,No results,No results,No results,"2400 Creekview Dr., Waco, TX 76711"
332,"Brian Ringler Associates, Carpenter, Petersen ...",No results,No results,No results,No results,"ATM, ATM, Cardtronics ATM, Wells Fargo ATM, Ba...","Walmart Supercenter, Baskin-Robbins, Dunkin', ...","Wells Fargo Bank, Bank of America (with Drive-...","Outback Steakhouse, Red Robin Gourmet Burgers ...","EB Medical Spa & Laser Center, Fantastic Sams ...",...,No results,No results,"Deceptive Games, Baker Ranch

## load
<br>

<div style="width: 100%; border-bottom: 1px solid black;"></div>

Once the data is extracted from the api it will be saved as a csv for modeling.

In [13]:
file = 'C:\\Users\\Owner\\OneDrive\\Desktop\\Work\\Theater\\TheaterFeatures.csv'

In [14]:
theater_features.to_csv(file, index=False)